# OCLC Download MARC

This script adds the MARC xml blob from the OCLC API to a TSV file, it is a preparatory script to run other scripts to extract metadata from the MARC data. 

It is a requirement that you have either the OCLC number in the data 

This script modifies the TSV file itself in batches, should the script timeout or other error you can rerun it and it will pickup where it left off, always run it on a backup of your orginal data files.

It creates a new column in the file `oclc_marc` which holds the MARC XML 

In [ ]:
import pandas as pd
import requests
import time


## Config
Set these variables below based on your setup

`path_to_tsv` - the path to the TSV file you want to run it on

`id_column_name` - the name of the column header that contains the oclc record id number

`user_agent` - this is the value put into the headers on each request, it is good practice to identifiy your client/project when working with open free APIs

`pause_between_req` - number of seconds to wait between each API call

`WSKey` - the OCLC WSkey

In [ ]:
path_to_tsv = "/Users/m/Downloads/data-tmp/hathitrust_post45fiction_metadata.tsv"
id_column_name = "oclc"
user_agent = 'YOUR PROJECT NAME HERE'
pause_between_req = 1
WSkey = "YOUR OCLC WSKEY HERE"


In [ ]:
def add_oclc(d):



    # if there is already a value skip it
    if 'oclc_marc' in d:
        if type(d['oclc_marc']) == str:        
            print('Skip',d[id_column_name])
            return d

    # if you want to add any logic to only download some records add it here
    if type(d['author_lccn']) == str:
        print('Skip',d[id_column_name])
        return d

    try:
        int(d[id_column_name])
    except:
        print("Cannot convert to int",d[id_column_name])
        return d
    
    url = f"https://www.worldcat.org/webservices/catalog/content/{int(d[id_column_name])}?wskey={WSkey}"
    
    r = requests.get(url, headers={'Accept': 'application/json', 'User-Agent': user_agent})
    
    if '<leader>' not in r.text:
        print("No OCLC record XML:",d[id_column_name])
        return d
    
    d['oclc_marc'] = r.text

    time.sleep(pause_between_req)

    return d

In [ ]:
# load the tsv
df = pd.read_csv(path_to_tsv, sep='\t', header=0, low_memory=False)

# we are going to split the dataframe into chunks so we can save our progress as we go but don't want to save the entire file on on every record operation
n = 100  #chunk row size
list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]

# loop through each chunk
for idx, df_chunk in enumerate(list_df):

    # if you want it to skip X number of chunks uncomment this, the number is the row to skip to
    if idx < 537:
        continue

    print("Working on chunk ", idx, 'of', len(list_df))
    list_df[idx] = list_df[idx].apply(lambda d: add_oclc(d),axis=1 )  

    reformed_df = pd.concat(list_df)
    reformed_df.to_csv(path_to_tsv, sep='\t')


